In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [10]:
tag_users = pd.read_csv("data/tag_users.csv")
tag_questions = pd.read_csv("data/tag_questions.csv")
tags = pd.read_csv("data/tags.csv")
professionals = pd.read_csv("data/professionals.csv")
answers = pd.read_csv("data/answers.csv")
emails1 = pd.read_csv("data/emails1.csv")
emails2 = pd.read_csv("data/emails2.csv")
emails = pd.concat([emails1,emails2])
questions = pd.read_csv("data/questions.csv")
matches1 = pd.read_csv("data/matches1.csv")
matches2 = pd.read_csv("data/matches2.csv")
matches3 = pd.read_csv("data/matches3.csv")
matches = pd.concat([matches1,matches2,matches3])
answers_scores = pd.read_csv("data/answer_scores.csv")
group_members = pd.read_csv("data/group_memberships.csv")
schools_members = pd.read_csv("data/school_memberships.csv")

In [67]:
tag_questions

,tag_questions_tag_id,tag_questions_question_id
0,28930,cb43ebee01364c68ac61d347a393ae39
1,28930,47f55e85ce944242a5a347ab85a8ffb4
2,28930,ccc30a033a0f4dfdb2eb987012f25792
3,28930,e30b274e48d741f7bf50eb5e7171a3c0
4,28930,3d22742052df4989b311b4195cbb0f1a
...,...,...
76548,39183,72affe86909d49c09761a1d376476c92
76549,39248,5b86153f2b6c46a4801bb39e10e01281
76550,39249,0946a81d511b48c58376fd59058d3b12
76551,39249,b4de8ccfcd874481bfc80ceacb0caac8


In [11]:
def get_user_tags(user_id):
    user_tags = tag_users[tag_users["tag_users_user_id"] == user_id]["tag_users_tag_id"]
    tags_names =tags.loc[tags["tags_tag_id"].isin(user_tags)]
    return(tags)

In [12]:
def get_user_answers(user_id):
    all_answers = answers.loc[answers["answers_author_id"]==user_id]
    return all_answers

In [13]:
def match_email_question(email_date):
    f = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S UTC+0000")
    b  =f(email_date)
    t = questions["questions_date_added"].apply(f)
    return questions.iloc[(b-t).abs().argsort()[:1]]

In [14]:
immediate_emails = emails[emails["emails_frequency_level"]=="email_notification_immediate"]
immediate_matches = matches[matches["matches_email_id"].isin(immediate_emails["emails_id"])]
immediate_emails = immediate_emails[immediate_emails["emails_id"].isin(matches["matches_email_id"])]

immediate_emails = immediate_emails.rename(columns={'emails_id':"id"})
immediate_matches = immediate_matches.rename(columns={'matches_email_id':"id"})

In [15]:
emails_questions = pd.merge(immediate_emails, immediate_matches, how="left", on="id")
emails_questions = emails_questions.rename(columns = {"matches_question_id": "question_id", "emails_recipient_id": "professional_id"})
answers = answers.rename(columns = {"answers_question_id": "question_id", "answers_author_id": "professional_id"})

In [16]:
full_data = pd.merge(emails_questions, answers,how='left',  on=["professional_id","question_id" ])
target = full_data['answers_id'].fillna(0)
target = pd.to_numeric(target, errors='coerce').fillna(1).astype(int)
full_data["q_answered"] = target
full_data = full_data.rename(columns = {"id": "email_id"})
date_vectorizer = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S UTC+0000")
full_data["emails_date_sent"] = full_data["emails_date_sent"].apply(date_vectorizer)
indices = full_data["answers_date_added"][full_data["answers_date_added"].notnull()].index.values
full_data.loc[indices, "answers_date_added"] = full_data["answers_date_added"][full_data["answers_date_added"].notnull()].apply(date_vectorizer)
full_data["time_taken"] =  pd.to_datetime(full_data["answers_date_added"]) - pd.to_datetime(full_data["emails_date_sent"])
indices2 = full_data["time_taken"][full_data["time_taken"] < datetime.timedelta( days=0)].index.values
full_data.loc[indices2, "time_taken"]=datetime.timedelta(days=0, seconds=0,minutes=0, hours=0 )

In [17]:
response = full_data.groupby("professional_id").mean().drop(columns = ["email_id"]).reset_index()
professionals_dataset = pd.DataFrame(columns = ["professional_id"])
professionals_dataset["professional_id"] = professionals["professionals_id"]
professionals_dataset = pd.merge(professionals_dataset, response,how='left',  on=["professional_id" ])
professionals_dataset = professionals_dataset.rename(columns = {"q_answered": "response_rate"})
grouped = full_data.groupby('professional_id')["time_taken"]
time_mean = grouped.apply(lambda x: np.mean(x))
professionals_dataset = pd.merge(professionals_dataset, time_mean,how='left',  on=["professional_id" ])
professionals_dataset=  professionals_dataset.rename(columns ={"time_taken": "avg_time_taken"})
answers_count = answers["professional_id"].value_counts().reset_index().rename(columns = {"professional_id":"number_q_answered","index":"professional_id",  })
professionals_dataset = pd.merge(professionals_dataset, answers_count,how='left',  on=["professional_id" ])
all_tags = pd.merge(tag_users.rename(columns = {"tag_users_tag_id": "tag_id"}),tags.rename(columns = {"tags_tag_id": "tag_id"}),how='left',  on=["tag_id" ])
foll_tags = all_tags.groupby('tag_users_user_id')['tags_tag_name'].apply(list).reset_index(name='following_tags').rename(columns = {"tag_users_user_id": "professional_id"})
professionals_dataset = pd.merge(professionals_dataset, foll_tags,how='left',  on=["professional_id" ])
a = pd.merge(answers["question_id"].reset_index(), tag_questions.rename(columns = {"tag_questions_question_id": "question_id"}),how='left',  on=["question_id" ])
b = pd.merge(a.rename(columns = {"tag_questions_tag_id":"tags_tag_id"}), tags,how='left',  on=["tags_tag_id" ]).drop(columns = ["index", "tags_tag_id"])
c = pd.merge(b, answers,how='left',  on=["question_id" ]).drop(columns = ["answers_id", "answers_date_added", "answers_body", "question_id"])
d = c.groupby("professional_id")["tags_tag_name"].apply(list).reset_index()
professionals_dataset = pd.merge(professionals_dataset, d.rename(columns = {"tags_tag_name": "prev_q_tags"}),how='left',  on=["professional_id" ])
prof_score = pd.merge(answers[["professional_id","answers_id" ]], answers_scores.rename(columns = {"id":"answers_id"}),how='left',  on=["answers_id" ]).drop(columns = ["answers_id"])
prof_score.groupby("professional_id").mean().reset_index()
professionals_dataset = pd.merge(professionals_dataset,prof_score,how='left',  on=["professional_id" ]).rename(columns = {"score":"avg_ansrs_score"})
prof_grp = group_members["group_memberships_user_id"].value_counts().reset_index().rename(columns = {"index":"professional_id", "group_memberships_user_id":"num_groups"})
professionals_dataset = pd.merge(professionals_dataset,prof_grp,how='left',  on=["professional_id" ])
prof_schl = schools_members["school_memberships_user_id"].value_counts().reset_index().rename(columns = {"index":"professional_id", "school_memberships_user_id":"num_schools"})
professionals_dataset = pd.merge(professionals_dataset,prof_schl,how='left',  on=["professional_id" ])
prof_emls = immediate_emails["emails_recipient_id"].value_counts().reset_index().rename(columns = {"index":"professional_id", "emails_recipient_id":"num_emails"})
professionals_dataset = pd.merge(professionals_dataset,prof_emls,how='left',  on=["professional_id" ])
professionals_dataset["answrs_emails_ratio"] = professionals_dataset["number_q_answered"] / professionals_dataset["num_emails"]


In [18]:
professionals_dataset

,professional_id,Unnamed: 0_x,Unnamed: 0_y,response_rate,avg_time_taken,number_q_answered,following_tags,prev_q_tags,avg_ansrs_score,num_groups,num_schools,num_emails,answrs_emails_ratio
0,9ced4ce7519049c0944147afb75a8ce3,NaN,NaN,NaN,NaT,1.0,NaN,"[consulting, resume, consulting, resume, consu...",5.0,NaN,NaN,NaN,NaN
1,f718dcf6d2ec4cb0a52a9db59d7f9e67,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0c673e046d824ec0ad0ebe012a0673e4,329.342857,2.402384e+06,0.0,NaT,39.0,"[consulting, education, consulting, education,...","[experience, healthcare, doctor, anesthesiolog...",1.0,NaN,NaN,70.0,0.557143
3,0c673e046d824ec0ad0ebe012a0673e4,329.342857,2.402384e+06,0.0,NaT,39.0,"[consulting, education, consulting, education,...","[experience, healthcare, doctor, anesthesiolog...",0.0,NaN,NaN,70.0,0.557143
4,0c673e046d824ec0ad0ebe012a0673e4,329.342857,2.402384e+06,0.0,NaT,39.0,"[consulting, education, consulting, education,...","[experience, healthcare, doctor, anesthesiolog...",3.0,NaN,NaN,70.0,0.557143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68186,9f267950ab8e43e2afcf564327049297,NaN,NaN,NaN,NaT,NaN,[broadcast-media],NaN,NaN,NaN,NaN,NaN,NaN
68187,4a7e1e5dd884488da283e1cab6ad11e0,NaN,NaN,NaN,NaT,NaN,"[telecommunications, firstnet, #inbuildingsolu...",NaN,NaN,NaN,NaN,NaN,NaN
68188,ea75c5fce38348e0a151c3c346929e6a,NaN,NaN,NaN,NaT,NaN,"[computer-software, electrical-engineering, ha...",NaN,NaN,NaN,NaN,NaN,NaN
68189,34f6222c3c02480ca2df8a3e4ba878d0,NaN,NaN,NaN,NaT,NaN,[telecommunications],NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
emails.head(1)

,Unnamed: 0,emails_id,emails_recipient_id,emails_date_sent,emails_frequency_level
0,0,2337714,0c673e046d824ec0ad0ebe012a0673e4,2018-12-07 01:05:40 UTC+0000,email_notification_daily


In [28]:
match_proff = pd.merge(matches.rename(columns = {"matches_email_id":"emails_id"}), emails,how='left',  on=["emails_id" ])


In [31]:
match_proff

,Unnamed: 0_x,emails_id,matches_question_id,Unnamed: 0_y,emails_recipient_id,emails_date_sent,emails_frequency_level
0,0,1721939,332a511f1569444485cf7a7a556a5e54,1303869,172d6cac9efd4f41929430b163854b6b,2018-03-25 01:03:06 UTC+0000,email_notification_daily
1,1,1665388,332a511f1569444485cf7a7a556a5e54,1257634,430c9844fa3c410d88bb4c9e2fb95e99,2018-03-08 01:18:13 UTC+0000,email_notification_daily
2,2,1636634,332a511f1569444485cf7a7a556a5e54,1559127,82d34dd2bf8d4364b49c748080a4145f,2018-02-28 02:26:09 UTC+0000,email_notification_daily
3,3,1635498,332a511f1569444485cf7a7a556a5e54,1321319,21d954e0bb414a74adb126fa846d0255,2018-02-28 01:23:45 UTC+0000,email_notification_daily
4,4,1620298,332a511f1569444485cf7a7a556a5e54,1575696,df59997f4576482a9af22dbce2e2628b,2018-02-24 00:41:01 UTC+0000,email_notification_daily
...,...,...,...,...,...,...,...
4316270,4316270,184809,1b1ebe9afc5046ccbb328704e88e5cc0,251065,4172e22f896049dba8c83311c78b1545,2015-10-23 21:00:11 UTC+0000,email_notification_weekly
4316271,4316271,184814,1b1ebe9afc5046ccbb328704e88e5cc0,343708,4c321ecdb7f84364a83a94afdfd88866,2015-10-23 21:00:13 UTC+0000,email_notification_weekly
4316272,4316272,184815,1b1ebe9afc5046ccbb328704e88e5cc0,386768,f411003a039e478996686160dd13a260,2015-10-23 21:00:13 UTC+0000,email_notification_weekly
4316273,4316273,184818,1b1ebe9afc5046ccbb328704e88e5cc0,397706,a87269ea441f4218af51fe27979657cc,2015-10-23 21:00:14 UTC+0000,email_notification_weekly


In [45]:
matches_questions = pd.merge(answers, match_proff.rename(columns = {"emails_recipient_id":"professional_id" }),how='left',  on=["professional_id" ])


In [46]:
answers.head(3)

,answers_id,professional_id,question_id,answers_date_added,answers_body
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...
1,ada720538c014e9b8a6dceed09385ee3,2aa47af241bf42a4b874c453f0381bd4,eb80205482e4424cad8f16bc25aa2d9c,2018-05-01 14:19:08 UTC+0000,<p>Hi. I joined the Army after I attended coll...
2,eaa66ef919bc408ab5296237440e323f,cbd8f30613a849bf918aed5c010340be,eb80205482e4424cad8f16bc25aa2d9c,2018-05-02 02:41:02 UTC+0000,"<p>Dear Priyanka,</p><p>Greetings! I have answ..."


In [47]:
match_proff[match_proff.emails_recipient_id == '36ff3b3666df400f956f8335cf53e09e'].head(3)

,Unnamed: 0_x,emails_id,matches_question_id,Unnamed: 0_y,emails_recipient_id,emails_date_sent,emails_frequency_level
168,168,306328,332a511f1569444485cf7a7a556a5e54,546074,36ff3b3666df400f956f8335cf53e09e,2016-04-26 11:15:09 UTC+0000,email_notification_immediate
1818,1818,493142,0f1d6a4f276c4a05878dd48e03e52289,545521,36ff3b3666df400f956f8335cf53e09e,2016-05-19 22:17:12 UTC+0000,email_notification_immediate
3995,3995,2006235,be0bac33e0594b21ac1f82fe2ffac7fb,543930,36ff3b3666df400f956f8335cf53e09e,2018-07-09 00:01:07 UTC+0000,email_notification_immediate


In [49]:
matches_questions.head(5)

,answers_id,professional_id,question_id,answers_date_added,answers_body,Unnamed: 0_x,emails_id,matches_question_id,Unnamed: 0_y,emails_date_sent,emails_frequency_level
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...,168.0,306328.0,332a511f1569444485cf7a7a556a5e54,546074.0,2016-04-26 11:15:09 UTC+0000,email_notification_immediate
1,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...,1818.0,493142.0,0f1d6a4f276c4a05878dd48e03e52289,545521.0,2016-05-19 22:17:12 UTC+0000,email_notification_immediate
2,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...,3995.0,2006235.0,be0bac33e0594b21ac1f82fe2ffac7fb,543930.0,2018-07-09 00:01:07 UTC+0000,email_notification_immediate
3,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...,7358.0,1837294.0,f05b579f2cd3432898f3ba54feee86b6,543379.0,2018-04-24 16:43:14 UTC+0000,email_notification_immediate
4,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...,9088.0,1787837.0,0149c6d63e214040b44d4a3789bb00ba,543751.0,2018-04-12 17:13:53 UTC+0000,email_notification_immediate


In [50]:
matches_questions[["answers_id", "professional_id","question_id","emails_id" ]].head(5)

,answers_id,professional_id,question_id,emails_id
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,306328.0
1,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,493142.0
2,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2006235.0
3,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1837294.0
4,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1787837.0


In [26]:
matches_questions["emails_id"] = matches_questions["emails_id"].fillna(0)
indices = matches_questions["emails_id"][matches_questions["emails_id"]>0].index.values
matches_questions.loc[indices,"emails_id" ]=1

In [27]:
matches_questions.rename(columns = {"emails_id": "is_matched"})

,answers_id,professional_id,question_id,is_matched
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1.0
1,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1.0
2,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1.0
3,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1.0
4,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1.0
...,...,...,...,...
26521409,563bc22e715e42bfb4b6ef9c3a95fb23,5750e3d592cd4b0486ec4e7eb01c29ad,1b1ebe9afc5046ccbb328704e88e5cc0,1.0
26521410,563bc22e715e42bfb4b6ef9c3a95fb23,5750e3d592cd4b0486ec4e7eb01c29ad,1b1ebe9afc5046ccbb328704e88e5cc0,1.0
26521411,563bc22e715e42bfb4b6ef9c3a95fb23,5750e3d592cd4b0486ec4e7eb01c29ad,1b1ebe9afc5046ccbb328704e88e5cc0,1.0
26521412,563bc22e715e42bfb4b6ef9c3a95fb23,5750e3d592cd4b0486ec4e7eb01c29ad,1b1ebe9afc5046ccbb328704e88e5cc0,1.0


In [54]:
answers_matched = pd.merge(answers, match_proff, how='left', left_on=["professional_id", "question_id"], right_on = ['emails_recipient_id','matches_question_id'])

In [57]:
answers_matched['is_matched'] = answers_matched['emails_id'].fillna(0)

In [58]:
answers_matched

,answers_id,professional_id,question_id,answers_date_added,answers_body,Unnamed: 0_x,emails_id,matches_question_id,Unnamed: 0_y,emails_recipient_id,emails_date_sent,emails_frequency_level,is_matched
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...,168.0,306328.0,332a511f1569444485cf7a7a556a5e54,546074.0,36ff3b3666df400f956f8335cf53e09e,2016-04-26 11:15:09 UTC+0000,email_notification_immediate,306328.0
1,ada720538c014e9b8a6dceed09385ee3,2aa47af241bf42a4b874c453f0381bd4,eb80205482e4424cad8f16bc25aa2d9c,2018-05-01 14:19:08 UTC+0000,<p>Hi. I joined the Army after I attended coll...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,eaa66ef919bc408ab5296237440e323f,cbd8f30613a849bf918aed5c010340be,eb80205482e4424cad8f16bc25aa2d9c,2018-05-02 02:41:02 UTC+0000,"<p>Dear Priyanka,</p><p>Greetings! I have answ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,1a6b3749d391486c9e371fbd1e605014,7e72a630c303442ba92ff00e8ea451df,4ec31632938a40b98909416bdd0decff,2017-05-10 19:00:47 UTC+0000,<p>I work for a global company who values high...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,5229c514000446d582050f89ebd4e184,17802d94699140b0a0d2995f30c034c6,2f6a9a99d9b24e5baa50d40d0ba50a75,2017-10-13 22:07:33 UTC+0000,I agree with Denise. Every single job I've had...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51186,74d708859d7e4d8c9d330506733e1fc3,b08b41c9e3724b609930779875b6b284,b64ef9cf0d33475ca9c22bdb8749f5a8,2016-10-01 08:42:28 UTC+0000,<p>Careers in law enforcement have different r...,4316055.0,803127.0,b64ef9cf0d33475ca9c22bdb8749f5a8,683197.0,b08b41c9e3724b609930779875b6b284,2016-10-01 00:02:15 UTC+0000,email_notification_daily,803127.0
51187,2070acba9f89433fa7dd9bcdc5f12351,f1ca9370286248aba5aae737f92a07dd,1b1ebe9afc5046ccbb328704e88e5cc0,2015-10-22 00:06:49 UTC+0000,<p>Well straight up I'm going to tell you it's...,4316256.0,179444.0,1b1ebe9afc5046ccbb328704e88e5cc0,501789.0,f1ca9370286248aba5aae737f92a07dd,2015-10-21 20:17:55 UTC+0000,email_notification_immediate,179444.0
51188,322bc9269c47467cbbee4f2b6b685ae2,9798c8df06244283ba14d30978ec318a,1b1ebe9afc5046ccbb328704e88e5cc0,2015-10-21 21:52:17 UTC+0000,"<p>When I was in my late thirties, with a bach...",4316105.0,179278.0,1b1ebe9afc5046ccbb328704e88e5cc0,436542.0,9798c8df06244283ba14d30978ec318a,2015-10-21 20:16:47 UTC+0000,email_notification_immediate,179278.0
51189,e554bc001ece4cda9ef9524a83239fbc,597cd583fb1049deb9b68ce308a992a8,1b1ebe9afc5046ccbb328704e88e5cc0,2015-10-21 20:22:39 UTC+0000,<p>I dont think age is a criteria at all. Expe...,4316157.0,179336.0,1b1ebe9afc5046ccbb328704e88e5cc0,177973.0,597cd583fb1049deb9b68ce308a992a8,2015-10-21 20:17:11 UTC+0000,email_notification_immediate,179336.0


In [60]:
answers_matched.is_matched = answers_matched.is_matched.apply(lambda x: int(x>0))

In [62]:
answers_matched.columns

Index(['answers_id', 'professional_id', 'question_id', 'answers_date_added',
       'answers_body', 'Unnamed: 0_x', 'emails_id', 'matches_question_id',
       'Unnamed: 0_y', 'emails_recipient_id', 'emails_date_sent',
       'emails_frequency_level', 'is_matched'],
      dtype='object')

In [64]:
answers_matched

,answers_id,professional_id,question_id,answers_date_added,answers_body,Unnamed: 0_x,emails_id,matches_question_id,Unnamed: 0_y,emails_recipient_id,emails_date_sent,emails_frequency_level,is_matched
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...,168.0,306328.0,332a511f1569444485cf7a7a556a5e54,546074.0,36ff3b3666df400f956f8335cf53e09e,2016-04-26 11:15:09 UTC+0000,email_notification_immediate,1
1,ada720538c014e9b8a6dceed09385ee3,2aa47af241bf42a4b874c453f0381bd4,eb80205482e4424cad8f16bc25aa2d9c,2018-05-01 14:19:08 UTC+0000,<p>Hi. I joined the Army after I attended coll...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,eaa66ef919bc408ab5296237440e323f,cbd8f30613a849bf918aed5c010340be,eb80205482e4424cad8f16bc25aa2d9c,2018-05-02 02:41:02 UTC+0000,"<p>Dear Priyanka,</p><p>Greetings! I have answ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1a6b3749d391486c9e371fbd1e605014,7e72a630c303442ba92ff00e8ea451df,4ec31632938a40b98909416bdd0decff,2017-05-10 19:00:47 UTC+0000,<p>I work for a global company who values high...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5229c514000446d582050f89ebd4e184,17802d94699140b0a0d2995f30c034c6,2f6a9a99d9b24e5baa50d40d0ba50a75,2017-10-13 22:07:33 UTC+0000,I agree with Denise. Every single job I've had...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51186,74d708859d7e4d8c9d330506733e1fc3,b08b41c9e3724b609930779875b6b284,b64ef9cf0d33475ca9c22bdb8749f5a8,2016-10-01 08:42:28 UTC+0000,<p>Careers in law enforcement have different r...,4316055.0,803127.0,b64ef9cf0d33475ca9c22bdb8749f5a8,683197.0,b08b41c9e3724b609930779875b6b284,2016-10-01 00:02:15 UTC+0000,email_notification_daily,1
51187,2070acba9f89433fa7dd9bcdc5f12351,f1ca9370286248aba5aae737f92a07dd,1b1ebe9afc5046ccbb328704e88e5cc0,2015-10-22 00:06:49 UTC+0000,<p>Well straight up I'm going to tell you it's...,4316256.0,179444.0,1b1ebe9afc5046ccbb328704e88e5cc0,501789.0,f1ca9370286248aba5aae737f92a07dd,2015-10-21 20:17:55 UTC+0000,email_notification_immediate,1
51188,322bc9269c47467cbbee4f2b6b685ae2,9798c8df06244283ba14d30978ec318a,1b1ebe9afc5046ccbb328704e88e5cc0,2015-10-21 21:52:17 UTC+0000,"<p>When I was in my late thirties, with a bach...",4316105.0,179278.0,1b1ebe9afc5046ccbb328704e88e5cc0,436542.0,9798c8df06244283ba14d30978ec318a,2015-10-21 20:16:47 UTC+0000,email_notification_immediate,1
51189,e554bc001ece4cda9ef9524a83239fbc,597cd583fb1049deb9b68ce308a992a8,1b1ebe9afc5046ccbb328704e88e5cc0,2015-10-21 20:22:39 UTC+0000,<p>I dont think age is a criteria at all. Expe...,4316157.0,179336.0,1b1ebe9afc5046ccbb328704e88e5cc0,177973.0,597cd583fb1049deb9b68ce308a992a8,2015-10-21 20:17:11 UTC+0000,email_notification_immediate,1


In [65]:
answers_data = answers_matched[['answers_id', 'professional_id', 'question_id', 'answers_date_added',
       'answers_body', 'Unnamed: 0_x', 'emails_id', 'matches_question_id',
       'Unnamed: 0_y', 'emails_recipient_id', 'emails_date_sent',
       'emails_frequency_level', 'is_matched']]